In [27]:
import urllib

In [28]:
!wget https://raw.githubusercontent.com/karpathy/ng-video-lecture/master/input.txt

In [29]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [30]:
len(text)

1115393

In [31]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [32]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [33]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [34]:
print(encode('saluu'))
print(decode(encode('saluu')))

[57, 39, 50, 59, 59]
saluu


In [35]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
data[:36]

torch.Size([1115393]) torch.int64


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63])

In [36]:
# Train/Validation dataset split
n = int(0.9 * len(data))
data_train = data[:n]
data_val = data[n:]

In [37]:
block_size = 8
x = data_train[:block_size]
y = data_train[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'Context: {context} | Target: {target}')

Context: tensor([18]) | Target: 47
Context: tensor([18, 47]) | Target: 56
Context: tensor([18, 47, 56]) | Target: 57
Context: tensor([18, 47, 56, 57]) | Target: 58
Context: tensor([18, 47, 56, 57, 58]) | Target: 1
Context: tensor([18, 47, 56, 57, 58,  1]) | Target: 15
Context: tensor([18, 47, 56, 57, 58,  1, 15]) | Target: 47
Context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) | Target: 58


In [38]:
# Data batching
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = data_train if split == 'train' else data_val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

In [39]:
xb, yb = get_batch('train')
xb

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])